## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-14-01-13-00 +0000,wsj,A growing number of blue cities and states acr...,https://www.wsj.com/economy/democratic-states-...
1,2025-08-14-01-12-48 +0000,bbc,What we learned from Taylor Swift's highly ant...,https://www.bbc.com/news/articles/cedv5dy9v8lo...
2,2025-08-14-01-00-00 +0000,wsj,Democrats are struggling to recover support fr...,https://www.wsj.com/politics/policy/silicon-va...
3,2025-08-14-01-00-00 +0000,wsj,The Secret Channel Russia and Ukraine Use to T...,https://www.wsj.com/world/pow-trade-russia-ukr...
4,2025-08-14-01-00-00 +0000,wsj,How Finland’s Golf-Loving President Got Trump ...,https://www.wsj.com/world/finland-president-al...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
49,trump,60
542,putin,20
166,new,16
39,ukraine,11
68,killed,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
185,2025-08-13-15-03-41 +0000,nypost,Trump vows main goal of Putin Alaska summit is...,https://nypost.com/2025/08/13/world-news/trump...,116
144,2025-08-13-17-57-20 +0000,nypost,Trump says he doesn’t believe he can convince ...,https://nypost.com/2025/08/13/us-news/trump-sa...,116
124,2025-08-13-19-04-10 +0000,nyt,Trump’s Affinity for Putin Will Be Tested at H...,https://www.nytimes.com/2025/08/13/us/politics...,113
169,2025-08-13-16-25-00 +0000,wsj,New York Attorney General Letitia James is sui...,https://www.wsj.com/politics/policy/new-york-a...,113
223,2025-08-13-11-59-00 +0000,wsj,"Ahead of Trump-Putin Meeting, Zelensky Says He...",https://www.wsj.com/world/ahead-of-trump-putin...,111


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
185,116,2025-08-13-15-03-41 +0000,nypost,Trump vows main goal of Putin Alaska summit is...,https://nypost.com/2025/08/13/world-news/trump...
244,56,2025-08-13-10-00-00 +0000,nypost,Sean Duffy opens up on spat with Elon Musk ove...,https://nypost.com/2025/08/13/us-news/sean-duf...
169,53,2025-08-13-16-25-00 +0000,wsj,New York Attorney General Letitia James is sui...,https://www.wsj.com/politics/policy/new-york-a...
58,47,2025-08-13-21-56-17 +0000,nypost,NY man harassed slain UnitedHealthcare CEO Bri...,https://nypost.com/2025/08/13/us-news/ny-man-h...
277,43,2025-08-13-05-25-45 +0000,cbc,Air Canada says it will begin cancelling fligh...,https://www.cbc.ca/news/business/air-canada-un...
173,40,2025-08-13-16-03-42 +0000,nypost,"Trump announces George Strait, Sylvester Stall...",https://nypost.com/2025/08/13/us-news/trump-an...
230,37,2025-08-13-10-47-47 +0000,nypost,DC Mayor Muriel Bowser backtracks on support f...,https://nypost.com/2025/08/13/us-news/dc-mayor...
297,35,2025-08-13-01-49-00 +0000,wsj,E.J. Antoni lacks the research record of previ...,https://www.wsj.com/politics/policy/ej-antoni-...
274,35,2025-08-13-05-46-11 +0000,bbc,Israel bombards Gaza City as UK and allies dem...,https://www.bbc.com/news/articles/clyj0dd0qj9o...
56,33,2025-08-13-22-03-29 +0000,nypost,NJ parents applaud new law that could send the...,https://nypost.com/2025/08/13/us-news/nj-paren...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
